In [ ]:
# UDP Broadcast Client using selectors (non-blocking)
# Run this in a separate Jupyter notebook

import socket
import json
import datetime
import uuid
import time
import selectors
import sys

# Create a unique ID for this client
client_id = str(uuid.uuid4())[:8]

# Create a selector
sel = selectors.DefaultSelector()

# Create UDP socket for receiving broadcasts
client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM, socket.IPPROTO_UDP)
client_socket.setsockopt(socket.SOL_SOCKET, socket.SO_BROADCAST, 1)
client_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)

# Bind to the client port to receive broadcasts
client_port = 37020
client_socket.bind(('', client_port))  # Empty string means all interfaces

# Create a socket for sending responses back to the server
response_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

# Store received messages for demonstration purposes
received_messages = []
last_processed_id = -1

# Print initialization message
print(f"Client {client_id} started, listening on port {client_port}")
print("Press Ctrl+C to stop")

# Function to handle received data
def handle_received_data(sock, mask):
    try:
        data, addr = sock.recvfrom(1024)
        server_ip = addr[0]
        server_port = addr[1]
        
        # Decode and parse the message
        message_str = data.decode('utf-8')
        message = json.loads(message_str)
        
        # Extract timestamp and state
        timestamp = message.get("timestamp", "unknown")
        state = message.get("state", "unknown")
        message_id = message.get("message_id", -1)
        
        # Get current reference to global variable
        global last_processed_id
        
        # Only process new messages (in case of duplicates)
        if message_id > last_processed_id:
            last_processed_id = message_id
            
            # Get current time
            receive_time = datetime.datetime.now().isoformat()
            
            # Print received message
            print(f"\nReceived broadcast from {server_ip}:{server_port}")
            print(f"Message ID: {message_id}")
            print(f"Timestamp: {timestamp}")
            print(f"State: {state}")
            
            # Append to message history
            received_messages.append({
                "server_ip": server_ip,
                "server_port": server_port,
                "timestamp": timestamp,
                "receive_time": receive_time,
                "state": state,
                "message_id": message_id
            })
            
            # Send response back to the server
            response = f"Client {client_id} received message {message_id}"
            response_socket.sendto(response.encode('utf-8'), (server_ip, server_port))
            
            # Print statistics
            print(f"Total messages received: {len(received_messages)}")
            
            # Display the last 5 states for demonstration
            if len(received_messages) >= 5:
                recent_states = [msg["state"] for msg in received_messages[-5:]]
                print(f"Last 5 states: {recent_states}")
    
    except json.JSONDecodeError:
        print(f"Error: Received invalid JSON data")
    
    except Exception as e:
        print(f"Error: {e}")

# Register socket with the selector
sel.register(client_socket, selectors.EVENT_READ, handle_received_data)

# Main event loop
try:
    # Indicator for user that we're waiting for messages
    waiting_indicator = ['|', '/', '-', '\\']
    indicator_index = 0
    last_indicator_time = time.time()
    
    while True:
        # Check for events with a small timeout (50ms)
        events = sel.select(timeout=0.05)
        for key, mask in events:
            callback = key.data
            callback(key.fileobj, mask)

        # Update waiting indicator every 0.5 seconds if no events
        current_time = time.time()
        if current_time - last_indicator_time > 0.5:
            sys.stdout.write(f"\rWaiting for broadcasts {waiting_indicator[indicator_index]} ")
            sys.stdout.flush()
            indicator_index = (indicator_index + 1) % len(waiting_indicator)
            last_indicator_time = current_time

except KeyboardInterrupt:
    print("\nClient shutting down...")

finally:
    # Clean up the sockets and selector
    sel.unregister(client_socket)
    sel.close()
    client_socket.close()
    response_socket.close()
    print("Client stopped")
    
    # Print summary
    print(f"\nSummary: Received {len(received_messages)} messages")